Following the video of Corey Schafer. 
https://www.youtube.com/watch?v=2Fp1N6dof0Y

In [ ]:
import subprocess

subprocess.run(cmd) will send the returned output to the print display when you run it. Note: you need to use proper command based on your OS. For example, dir vs ls (windows vs mac). Returncode 0 means no error.

In [8]:
subprocess.run('ls')

CompletedProcess(args='ls', returncode=0)

You can use the full cmd string as the argument. However, the follwing would give you an error.

In [5]:
subprocess.run('ls -la')

FileNotFoundError: [Errno 2] No such file or directory: 'ls -la': 'ls -la'

You need to specify shell=True when you use multiple arguments in your cmd.

In [6]:
subprocess.run('ls -la', shell=True)

CompletedProcess(args='ls -la', returncode=0)

However, using shell can be hazardous if used without knowledge or goes out of control, and also if user input is taken. Another way to execute cmds with arguments is to break down the full string in a list of strings.

In [7]:
subprocess.run(['ls','-la'])

CompletedProcess(args=['ls', '-la'], returncode=0)

If the returned output needs to be assigned to a variable: 

In [16]:
p1 = subprocess.run(['ls','-la'])  # this prints nothing as output directed to p1
print(p1.args)
print(p1.returncode)
print(p1.stdout)

['ls', '-la']
0
None


To get the returned output in stdout, set capture_output=True.

In [17]:
p1 = subprocess.run(['ls','-la'], capture_output=True)
print(p1.args)
print(p1.returncode)
print(p1.stdout)

['ls', '-la']
0
b'total 2344\ndrwxr-xr-x  6 wah333d  staff      192 Mar 21 13:04 .\ndrwxr-xr-x  6 wah333d  staff      192 Mar  2 23:19 ..\ndrwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints\n-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt\n-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb\n-rw-r--r--  1 wah333d  staff     8616 Mar 21 13:04 Learn Subprocess.ipynb\n'


To convert the stdout into string, one way is to decode it. Bytes converted to string.

In [18]:
print(p1.stdout.decode())

total 2344
drwxr-xr-x  6 wah333d  staff      192 Mar 21 13:04 .
drwxr-xr-x  6 wah333d  staff      192 Mar  2 23:19 ..
drwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints
-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt
-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb
-rw-r--r--  1 wah333d  staff     8616 Mar 21 13:04 Learn Subprocess.ipynb



Another way is to pass argument text=True in the subprocess.run. In the background, this puts the stdout and stderror directly in the subprocess.PIPE.

In [19]:
p1 = subprocess.run(['ls', '-la'], capture_output=True, text=True)
print(p1.stdout)

total 2344
drwxr-xr-x  6 wah333d  staff      192 Mar 21 13:08 .
drwxr-xr-x  6 wah333d  staff      192 Mar  2 23:19 ..
drwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints
-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt
-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb
-rw-r--r--  1 wah333d  staff    10352 Mar 21 13:08 Learn Subprocess.ipynb



To set the stdout argument directly in the subprocess.PIPE:

In [22]:
p1 = subprocess.run(['ls', '-la'], stdout=subprocess.PIPE, text=True)
print(p1.stdout)

total 2344
drwxr-xr-x  6 wah333d  staff      192 Mar 21 13:34 .
drwxr-xr-x  6 wah333d  staff      192 Mar  2 23:19 ..
drwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints
-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt
-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb
-rw-r--r--  1 wah333d  staff    11756 Mar 21 13:34 Learn Subprocess.ipynb



We can redirect the stdout directly to some files. 

In [24]:
with open('output.txt', 'w') as f:
    p1 = subprocess.run(['ls', '-la'], stdout=f, text=True)

What happens if there is error? Say we run the command to list files of a directory that does not exist.

In [27]:
p1 = subprocess.run(['ls', '-la', 'dne'], capture_output=True, text=True)  # prints nothing
print(p1.returncode)  # can be used as a conditional statement in your code
print(p1.stderr)

1
ls: dne: No such file or directory



We can also add check=True arguments to auto generate exception by python.

In [28]:
p1 = subprocess.run(['ls', '-la', 'dne'], capture_output=True, text=True, check=True)

CalledProcessError: Command '['ls', '-la', 'dne']' returned non-zero exit status 1.

The error can be ignored by redirecting to dev null. 

In [30]:
p1 = subprocess.run(['ls', '-la', 'dne'], stderr=subprocess.DEVNULL)
print(p1.stderr)

None


How to use one cmd's output to another cmd? 

In [34]:
p1 = subprocess.run(['cat', 'output.txt'], capture_output=True, text=True)
# print(p1.stdout)
p2 = subprocess.run(['grep', '-n', '.ipynb'], capture_output=True, text=True, input=p1.stdout)
print(p2.stdout)

4:drwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints
5:-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt
6:-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb
7:-rw-r--r--  1 wah333d  staff    12530 Mar 21 13:38 Learn Subprocess.ipynb



Using shell=True can be simple as following: 

In [35]:
p1 = subprocess.run('cat output.txt | grep -n .ipynb', capture_output=True, text=True, shell=True)
print(p1.stdout)

4:drwxr-xr-x  4 wah333d  staff      128 Mar 21 12:44 .ipynb_checkpoints
5:-rw-r--r--@ 1 wah333d  staff  1181248 Mar 17 14:13 COVID_19_analysis.ipynb.txt
6:-rw-r--r--  1 wah333d  staff      995 Dec 31 02:28 First.ipynb
7:-rw-r--r--  1 wah333d  staff    12530 Mar 21 13:38 Learn Subprocess.ipynb

